# Let's first install our modules

In [2]:
!pip install tangelo-gc

try:
    import os
except ImportError:
    print ("os error")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 598.9/598.9 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.3/288.3 kB 32.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 33.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 49.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pubchempy: filename=PubChemPy-1.0.4-py3-none-any.whl size=13820 sha256=4230b2e4811a4d137bcc8a79c5f5798d77079b0824577a949aeb2911772609af
  Stored in directory: /root/.cache/pip/wheels/90/7c/45/18a0671e3c3316966ef7ed9ad2b3f3300a7e41d3421a44e799
Successfully built pubchempy


In [3]:
!pip install PySCF

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.3/47.3 MB 9.9 MB/s eta 0:00:00


# Building frozen silicon hexagon

## First, we're building the shape. Let's use the automatic solver to get an idea of what the default freezing would be.

In [ ]:
##########################
from tangelo import SecondQuantizedMolecule as SQMol

# crystal lattice Si
# source: https://www.princeton.edu/~maelabs/mae324/glos324/silicon.htm
# si_xyz = [("Si", (0., 0., 0.)), ("Si", (0., 0., 2.35))]

# put into hexagon
si_xyz = [("Si", (0., 0., 0.)), ("Si", (0., -2.35, 0)), ("Si", (2.35*0.5, 2.35*0.866, 0)),
         ("Si", (2.35*0.5, -2.35*0.866, 0)), ("Si", (-2.35*0.5, 2.35*0.866, 0)), ("Si", (-2.35*0.5, -2.35*0.866, 0))]


si_6311gdp = SQMol(si_xyz, q=0, spin=2)
print(f"{si_6311gdp.n_active_mos} active molecular orbitals")
print(f"{si_6311gdp.n_active_electrons} active electrons")

##########################

basis_sets = [
    "STO-3G",       # Simple zeta, minimal basis.
    "3-21G",        # Double zeta.
    "6-31G",        # Double zeta with more Gaussian primitives.
    "6-31G(d,p)",   # Polarization functions (+ 5 d-orbitals for all atoms except H, +3 p-orbitals for H atoms) added.
    "6-311G(d,p)",  # Triple zeta with polarization functions.
    "6-311+G(d,p)", # Triple zeta with polarization functions and diffuse functions.
    "cc-pvqz",      # Quadruple zeta.
    "cc-pv5z"       # Quintuple zeta.
]

###
si_6311gdp.mo_occ

### Looking at the periodic table, to simplify the process, we can use just the electrons in the valence shell. We also want to minimize the number of orbitals, so we played around with the other frozen orbitals.

In [7]:
from tangelo import SecondQuantizedMolecule as SQMol
si6=  """Si 0.  0. 0.
         Si 0. -2.35 0.
         Si 2.35*0.5 2.35*0.866 0.
         Si 2.35*0.5 -2.35*0.866 0.
         Si -2.35*0.5 2.35*0.866 0.
         Si -2.35*0.5 -2.35*0.866 0. """

# freeze some orbitals
fo = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17,18, 19, 20, 21, 22, 23, 24]+[i for i in range(31,100)]

# Runs RHF calculation
mol_si6 = SQMol(si6, q=0, spin=0, basis='6-31g(d,p)', frozen_orbitals=fo, symmetry=True)

print("  #  Energy  Symm Occ")
for i in range(10):
    print(f"{i+1:3d}{mol_si6.mo_energies[i]: 9.4f}  {mol_si6.mo_symm_labels[i]}   {int(mol_si6.mo_occ[i])}")

# Active electrons, Active orbitals
print(f"Number of active electrons: {mol_si6.n_active_electrons}")
print(f"Number of active orbtials: {mol_si6.n_active_mos}")

  #  Energy  Symm Occ
  1 -68.9478  A1   2
  2 -68.8524  A1   2
  3 -68.8116  B2   2
  4 -68.8116  A1   2
  5 -68.8095  A1   2
  6 -68.8094  B2   2
  7  -6.4481  A1   2
  8  -6.2513  B2   2
  9  -6.2317  A1   2
 10  -6.2091  A1   2
Number of active electrons: 12
Number of active orbtials: 14


# Now, let us try to get the ground energy state. We will use FCI (Classical) and VQE (Combination) to compare the both.

In [ ]:
# Now, let us try to get the ground energy state.
print("Importing VQE, Ansatze")
from tangelo.algorithms.variational import VQESolver, BuiltInAnsatze
print("Importing FCI")
from tangelo.algorithms.classical import FCISolver

algorithm_resources = dict()

print("Solving FCI")
print(f"CASCI energy = {FCISolver(mol_si6).simulate()}")

# Ground state energy calculation with VQE, reference values with FCI
vqe_options = {"molecule": mol_si6, "ansatz": BuiltInAnsatze.UCCSD}
print("Solving")
vqe_solver = VQESolver(vqe_options)
print("Building")
vqe_solver.build()
vqe_energy = vqe_solver.simulate()
print("\n Ground Singlet state")
print(f"VQE energy = {vqe_energy}")

algorithm_resources["vqe_ground_state"] = vqe_solver.get_resources()

Importing VQE, Ansatze
Importing FCI
Solving FCI
CASCI energy = -1730.917060713716
Solving
Building


# The above cell did not run. But if we did, we could run the code below to get the different energy levels for the different excited states, calculate the deltas, and then use it to determine the range of absorbable energies, and thus frequencies and wavelengths.
#### We have this functional for Li2, if you want to check out that document.

In [ ]:
# After acvhieving the ground state, we can use deflation to get the first and subsequent excited states.
deflation_circuits = [vqe_solver.optimal_circuit.copy()]

# Calculate first and second excited states by adding optimal circuits to deflation_circuits
dict_vqe = {}
for i in range(3):
    vqe_options = {"molecule": mol_si6, "ansatz": BuiltInAnsatze.UpCCGSD,
                   "deflation_circuits": deflation_circuits, "deflation_coeff": 0.4}
    vqe_solver = VQESolver(vqe_options)
    vqe_solver.build()
    vqe_energy = vqe_solver.simulate()
    print(f"Excited state #{i+1} \t VQE energy = {vqe_energy}")
    algorithm_resources[f"vqe_deflation_state_{i+1}"] = vqe_solver.get_resources()
    dict_vqe[i+1] =  vqe_energy
    deflation_circuits.append(vqe_solver.optimal_circuit.copy())